# 3 Layer Convolutional Neural Network

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Constant Parameters
# Preprocessing Parameters
TRAIN_DIR = "../data/train/"
MODEL_DIR = "../models/"
VALIDATION_SPLIT = 0.2
IMAGE_HEIGHT = IMAGE_WIDTH = 128
SEED = 1

# Training Parameters
NUM_EPOCHS = 50
OPTIMIZER = keras.optimizers.Adam()
LOSS_FN = keras.losses.CategoricalCrossentropy()

In [3]:
# Model Architecture
def get_model(image_depth, num_classes):
    return keras.models.Sequential([
        # Conv Layer 1
        keras.layers.Conv2D(32, 3, strides = 2, activation = "relu", input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, image_depth)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Dropout(0.2),
        
        # Conv Layer 2
        keras.layers.Conv2D(64, 3, activation = "relu"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Dropout(0.2),
        
        # Conv Layer 3
        keras.layers.Conv2D(128, 3, activation = "relu"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size = (2, 2)),
        keras.layers.Dropout(0.2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation = "relu"),
        keras.layers.Dense(num_classes, activation = "softmax")
    ])

## Preprocessing - RGB + Augment

In [4]:
model_id = "3cnn_rgb_augment"              # Unique Identifier used for saving model
color_mode = "rgb"                        # "rgb", "rgba" or "grayscale"

# DO NOT CHANGE
if color_mode == "grayscale":
    image_depth = 1
elif color_mode == "rgb":
    image_depth = 3
elif color_mode == "rgba":
    image_depth = 4
else:
    raise Exception()

In [5]:
# Data Augmentation Parameters
image_generator = ImageDataGenerator(
    validation_split = VALIDATION_SPLIT,
    rescale = 1./255,
    rotation_range = 45,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.4
)

In [6]:
train_data_gen = image_generator.flow_from_directory(
    color_mode = color_mode,
    directory = TRAIN_DIR,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle = True,
    seed = SEED,
    class_mode = "categorical",
    subset = "training"
)

Found 84331 images belonging to 42 classes.


In [7]:
validation_data_gen = image_generator.flow_from_directory(
    color_mode = color_mode,
    directory = TRAIN_DIR,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode = "categorical",
    subset = "validation"
)

Found 21061 images belonging to 42 classes.


In [8]:
model = get_model(image_depth, train_data_gen.num_classes)

In [9]:
model.compile(
    optimizer = OPTIMIZER,
    loss = LOSS_FN,
    metrics = ["accuracy"]
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 63, 63, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 63, 63, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0

In [ ]:
history = model.fit(
    train_data_gen,
    epochs = NUM_EPOCHS,
    validation_data = validation_data_gen
)

Epoch 1/50
2636/2636 [==============================] - 1410s 535ms/step - loss: 3.4304 - accuracy: 0.1108 - val_loss: 3.2675 - val_accuracy: 0.1383
Epoch 2/50
2636/2636 [==============================] - 1329s 504ms/step - loss: 3.1680 - accuracy: 0.1627 - val_loss: 3.2269 - val_accuracy: 0.1541
Epoch 3/50
2636/2636 [==============================] - 1308s 496ms/step - loss: 3.0572 - accuracy: 0.1882 - val_loss: 3.3086 - val_accuracy: 0.1578
Epoch 4/50
2636/2636 [==============================] - 1282s 486ms/step - loss: 2.9802 - accuracy: 0.2090 - val_loss: 3.0817 - val_accuracy: 0.1898
Epoch 5/50
2636/2636 [==============================] - 1427s 541ms/step - loss: 2.9137 - accuracy: 0.2255 - val_loss: 2.8739 - val_accuracy: 0.2382
Epoch 6/50
2636/2636 [==============================] - 1262s 479ms/step - loss: 2.8626 - accuracy: 0.2391 - val_loss: 2.8542 - val_accuracy: 0.2447
Epoch 7/50
2636/2636 [==============================] - 1268s 481ms/step - loss: 2.8114 - accuracy: 0.2497

In [ ]:
acc = history.history["accuracy"]
loss = history.history["loss"]
validation_acc = history.history["val_accuracy"]
validation_loss = history.history["val_loss"]

epochs_range = range(NUM_EPOCHS)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, validation_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, validation_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
model.save(MODEL_DIR + "{}.h5".format(model_id))